We'll use the IMDB dataset, tf.keras for our model, Tensorflow Hub for transfer learning.

In [2]:
import numpy as np

import tensorflow as tf

!pip install -q tensorflow-hub
!pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print(f"Version: {tf.__version__}")
print(f"Eager version: {tf.executing_eagerly()}")
print(f"Hub version: {hub.__version__}")
print(f"GPU is {'available' if tf.config.experimental.list_physical_devices('GPU') else 'NOT AVAILABLE'}")

Version: 2.2.0
Eager version: True
Hub version: 0.8.0
GPU is NOT AVAILABLE


## Download the IMDB dataset

In [3]:
train_data, validation_data, test_data = tfds.load(
    name='imdb_reviews',
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
)

Shuffling and writing examples to /home/nxhuy/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteKLEU39/imdb_reviews-train.tfrecord


Shuffling and writing examples to /home/nxhuy/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteKLEU39/imdb_reviews-test.tfrecord


Shuffling and writing examples to /home/nxhuy/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteKLEU39/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /home/nxhuy/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


## Explore the data

In [5]:
# view data
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

# batch size = 10
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [6]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

## Build the model

In [7]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)

hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [9]:
model = tf.keras.Sequential([
    hub_layer,
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense_1 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


### Loss function and optimizer

In [10]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=['accuracy'])

## Train the model

In [11]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 3s 109ms/step - loss: 0.8532 - accuracy: 0.4954 - val_loss: 0.7327 - val_accuracy: 0.5499
Epoch 2/20
30/30 [==============================] - 3s 100ms/step - loss: 0.6643 - accuracy: 0.5889 - val_loss: 0.6254 - val_accuracy: 0.6310
Epoch 3/20
30/30 [==============================] - 3s 101ms/step - loss: 0.5961 - accuracy: 0.6493 - val_loss: 0.5844 - val_accuracy: 0.6602
Epoch 4/20
30/30 [==============================] - 3s 101ms/step - loss: 0.5592 - accuracy: 0.6857 - val_loss: 0.5539 - val_accuracy: 0.6888
Epoch 5/20
30/30 [==============================] - 3s 114ms/step - loss: 0.5267 - accuracy: 0.7185 - val_loss: 0.5264 - val_accuracy: 0.7171
Epoch 6/20
30/30 [==============================] - 3s 110ms/step - loss: 0.4956 - accuracy: 0.7452 - val_loss: 0.5004 - val_accuracy: 0.7404
Epoch 7/20
30/30 [==============================] - 3s 105ms/step - loss: 0.4639 - accuracy: 0.7695 - val_loss: 0.4751 - val_accuracy: 0.7587
Epoch 

## Evaluate the model

In [12]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

49/49 - 3s - loss: 0.3260 - accuracy: 0.8559
loss: 0.326
accuracy: 0.856


In [16]:
model.trainable_variables

[<tf.Variable 'Variable:0' shape=(20001, 20) dtype=float32, numpy=
 array([[-0.60565156, -0.01413231,  0.79470503, ...,  0.7346931 ,
         -0.341707  , -0.43011868],
        [-1.2047577 ,  0.21791467,  1.1384295 , ...,  0.74889004,
         -0.41222748,  0.11590002],
        [-1.1219835 , -0.08905376, -1.4896572 , ..., -0.76676595,
          0.962041  ,  0.30460244],
        ...,
        [ 0.13454673, -0.15173355,  0.5368477 , ...,  0.8319293 ,
         -0.61366147, -1.0842572 ],
        [ 0.20673902, -0.1418509 , -0.5856179 , ..., -0.02331898,
          1.6080966 , -0.6525672 ],
        [-0.01625557,  0.01267959,  0.0194213 , ..., -0.04215106,
         -0.05283974,  0.02751774]], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(20, 16) dtype=float32, numpy=
 array([[ 0.31555486,  0.11103451, -0.00966024,  0.1418405 , -0.22100939,
         -0.2745854 ,  0.10764332, -0.0221368 , -0.188964  ,  0.16571039,
          0.0438383 , -0.2725828 , -0.15375419, -0.16890861,  0.25942805

## References

- https://www.tensorflow.org/tutorials/keras/text_classification_with_hub